In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import norm
from scipy.special import expit as plogis

# p-dimensional isotonic regression.
def multiso(xmat, xpred):
    n, p = xmat.shape
    p -= 1
    ans = sparsebasis(xmat, p)
    amat0 = ans['amat']
    rank = ans['rank']
    m = len(amat0)
    y = ans['y']
    xmat = ans['xmat']
    amat = np.zeros((m, n))
    for i in range(m):
        amat[i, int(amat0[i, 0])] = -1
        amat[i, int(amat0[i, 1])] = 1
    ans = coneproj(y, amat)
    theta = ans['theta']
    df = ans['df']
    return {'theta': theta, 'df': df, 'xmat': xmat, 'y': y}

# get the edge vectors for the polar cone
def sparsebasis(xmat, p):
    n = xmat.shape[0]
    xrnk = np.arange(n)
    sm = 1e-10
    anew1 = np.zeros((int(n * (n - 1) / 2), 2))
    comp = np.zeros((n, n))
    nr = 0
    for i in range(n - 1):
        for j in range(i + 1, n):
            bigger = True
            for l in range(p):
                if xmat[i, l] > xmat[j, l] + sm:
                    bigger = False
                    break
            if bigger:
                nr += 1
                anew1[nr - 1, 0] = i
                anew1[nr - 1, 1] = j
                comp[i, j] = 1
    anew2 = anew1[:nr, :]
    dump = np.zeros(nr, dtype=bool)
    for i in range(n - 1):
        for j in range(i + 1, n):
            if comp[i, j] == 1 and j < n - 1:
                for k in range(j + 1, n):
                    if comp[j, k] == 1 and comp[i, k] == 1:
                        comp[i, k] = 2
    id = 0
    for i in range(n - 1):
        for j in range(i + 1, n):
            if comp[i, j] > sm:
                id += 1
                if comp[i, j] == 2:
                    dump[id - 1] = True
    anew3 = anew2[~dump, :]
    return {'amat': anew3, 'y': xmat[:, p], 'xmat': xmat[:, :p], 'rank': xrnk}

# coneproj function
def coneproj(y, amat):
    n, m = amat.shape
    sm = 1e-8
    h = np.zeros(m, dtype=bool)
    b2 = -amat @ y
    if np.max(b2) > sm:
        i = np.argmin(b2 == np.max(b2))
        h[i] = True
    else:
        check = True
        theta = np.zeros(n)
    while not check:
        xmat = amat[h, :]
        a = np.linalg.solve(xmat @ xmat.T, xmat @ y)
        if np.min(a) < -sm:
            avec = np.zeros(m)
            avec[h] = a
            i = np.argmin(avec == np.min(avec))
            h[i] = False
            check = False
        else:
            check = True
            theta = xmat.T @ a
            b2 = -amat @ (y - theta)
            if np.max(b2) > sm:
                i = np.argmin(b2 == np.max(b2))
                h[i] = True
                check = False
    bhat = y - theta
    return {'df': n - np.sum(h), 'theta': bhat}

# Kernel functions
def kepa(u):
    return 0.75 * (1 - u**2) * (np.abs(u) < 1)

def kgauss(u):
    return norm.pdf(u)